In [1]:
import pandas as pd

df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
df

,customer_id,firstname,lastname,record_number,days_since_opened,days_since_confirmed,primary_term,final_term,days_till_primary_close,days_till_final_close,...,encoded_payment_21,encoded_payment_22,encoded_payment_23,encoded_payment_24,encoded_loans_account_holder_type,encoded_loans_credit_status,encoded_loans_credit_type,encoded_loans_account_currency,primary_close_flag,final_close_flag
0,4814dfa2-45a8-46b9-8102-98ecbbec2d2a,Navya,Vasa,7,10,8,12,10,13,5,...,0.0,1.0,3.0,4.0,1,3,4,1,0,0
1,bb0abe41-cc89-4a1d-bb1c-48bd212ae00d,Azad,Warrior,5,15,6,7,6,9,1,...,3.0,3.0,3.0,4.0,1,3,4,1,0,0
2,edfd1a62-05da-4cfd-b8ae-521f05f0fd5f,Piya,Jha,1,15,17,8,5,9,1,...,3.0,3.0,3.0,4.0,1,3,4,1,0,0
3,3eaa4c2a-2316-48f4-b1b0-4aafe2286630,Rhea,Varma,11,7,2,11,13,14,8,...,3.0,3.0,3.0,4.0,1,2,4,1,0,0
4,c9a73239-baae-40eb-8855-dab07767ea86,Sahil,Dave,12,14,9,8,11,6,13,...,3.0,3.0,3.0,4.0,1,3,4,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106669,c341a4e3-b4e2-455b-80b2-2be2655e6e85,Gatik,Boase,19,1,5,8,11,2,6,...,3.0,3.0,3.0,4.0,1,3,5,1,0,0
1106670,29d8337c-13c9-4bb6-b794-762c386b1866,Jhanvi,Andra,10,5,8,4,3,5,7,...,3.0,3.0,3.0,4.0,1,3,5,1,0,0
1106671,e3b374f2-d96e-4c8b-901a-be80a8bea3cc,Lakshit,Kulkarni,3,8,7,13,0,4,9,...,3.0,3.0,3.0,4.0,1,3,4,1,0,0
1106672,b81e5b47-5c1b-4302-897c-2edc179d583f,Parinaaz,Chaudry,4,10,4,16,9,6,13,...,3.0,3.0,3.0,4.0,1,3,4,1,0,0


In [3]:
columns_with_null = df.columns[df.isnull().any()].tolist()
for cols in columns_with_null:
  mean = df[cols].mean()
  df[cols] = df[cols].fillna(mean)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1106674 entries, 0 to 1106673
Data columns (total 62 columns):
 #   Column                              Non-Null Count    Dtype  
---  ------                              --------------    -----  
 0   customer_id                         1106674 non-null  object 
 1   firstname                           1106674 non-null  object 
 2   lastname                            1106674 non-null  object 
 3   record_number                       1106674 non-null  int64  
 4   days_since_opened                   1106674 non-null  int64  
 5   days_since_confirmed                1106674 non-null  int64  
 6   primary_term                        1106674 non-null  int64  
 7   final_term                          1106674 non-null  int64  
 8   days_till_primary_close             1106674 non-null  int64  
 9   days_till_final_close               1106674 non-null  int64  
 10  loans_credit_limit                  1106674 non-null  int64  
 11  loans_next_

In [5]:
df = df.drop(['customer_id', 'firstname','lastname'], axis=1)

In [6]:
df['is_zero_utilization'] = pd.to_numeric(df['encoded_payment_1'],errors='coerce').astype(int)
df['is_zero_over_limit_count'] = pd.to_numeric(df['encoded_payment_1'],errors='coerce').astype(int)
df['is_zero_max_over_limit_count'] = pd.to_numeric(df['encoded_payment_1'],errors='coerce').astype(int)

In [7]:
df['is_zero_loans_over_90_days'] = pd.to_numeric(df['encoded_payment_1'],errors='coerce').astype(int)
df['is_zero_loans_within_60_to_90_days'] = pd.to_numeric(df['encoded_payment_23'],errors='coerce').astype(int)
df['is_zero_loans_within_30_to_60_days'] = pd.to_numeric(df['encoded_payment_23'],errors='coerce').astype(int)
df['is_zero_loans_within_5_to_30_days'] = pd.to_numeric(df['encoded_payment_23'],errors='coerce').astype(int)
df['is_zero_loans_within_5_days'] = pd.to_numeric(df['encoded_payment_23'],errors='coerce').astype(int)

In [8]:
import xgboost as xgb
import sklearn
from sklearn import model_selection
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.model_selection import train_test_split

In [9]:
X = df.drop(['primary_close_flag','final_close_flag'],axis=1)
y = df[['primary_close_flag','final_close_flag']]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
model = xgb.XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [13]:
y_pred = model.predict(X_test)
y_pred2 = model.predict(X_train)

In [14]:
accuracy = accuracy_score(y_test, y_pred)
acc1 = accuracy_score(y_train, y_pred2)
print(f'Accuracy test: {accuracy:.5f}')
print(f'Accuracy train: {acc1:.5f}')

Accuracy test: 0.99990
Accuracy train: 0.99998


In [15]:
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')
df1

,customer_id,firstname,lastname,record_number,days_since_opened,days_since_confirmed,primary_term,final_term,days_till_primary_close,days_till_final_close,...,encoded_payment_19,encoded_payment_20,encoded_payment_21,encoded_payment_22,encoded_payment_23,encoded_payment_24,encoded_loans_account_holder_type,encoded_loans_credit_status,encoded_loans_credit_type,encoded_loans_account_currency
0,1dffd171-010b-4773-894f-4095077b0e31,Aradhya,Deep,14,2,13,14,2,14,6,...,3.0,4.0,3.0,3.0,3.0,NaN,1,3,4,1
1,cf0e7f8c-b22b-410c-8ad6-4a5bfd4d11cf,Anahita,Anand,17,3,6,0,1,0,4,...,0.0,1.0,0.0,0.0,0.0,1.0,1,2,3,1
2,49c9f6ef-58f2-4334-bb44-9a4ded2e587f,Amani,Tak,14,9,10,14,7,10,4,...,3.0,4.0,3.0,3.0,3.0,4.0,1,2,4,1
3,daab9520-7716-4b1e-b791-907912a81f79,Tushar,Sodhi,7,14,4,12,11,6,13,...,3.0,4.0,3.0,3.0,3.0,4.0,1,3,4,1
4,9d4adc34-368e-4642-98a2-58d73b4294f7,Jivika,Rana,13,3,3,16,1,15,4,...,3.0,4.0,3.0,3.0,3.0,4.0,1,3,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474284,cd83731c-a9ec-47de-9071-36914c800f35,Yashvi,Saini,1,10,4,7,11,6,13,...,3.0,4.0,3.0,3.0,3.0,4.0,1,3,4,1
474285,562f1181-ad6e-467e-82fb-e38b443e9c3a,Ranbir,Sarna,8,3,11,7,3,11,12,...,3.0,4.0,3.0,3.0,3.0,4.0,1,3,5,1
474286,558ae4c7-e2d1-4e92-936d-ca0dda807e9f,Inaaya,Varghese,12,6,13,12,10,16,7,...,1.0,2.0,0.0,0.0,1.0,1.0,1,3,4,1
474287,26fa2f73-8c6d-4280-8610-13170a3062dc,Prisha,Kale,16,2,16,0,7,10,8,...,3.0,4.0,3.0,3.0,3.0,4.0,1,2,3,1


In [16]:
columns_with_null = df1.columns[df1.isnull().any()].tolist()
for cols in columns_with_null:
  mean = df1[cols].mean()
  df1[cols] = df1[cols].fillna(mean)

In [17]:
df1 = df1.drop(['customer_id', 'firstname','lastname'], axis=1)

In [18]:
df1['is_zero_utilization'] = pd.to_numeric(df1['encoded_payment_1'],errors='coerce').astype(int)
df1['is_zero_over_limit_count'] = pd.to_numeric(df1['encoded_payment_1'],errors='coerce').astype(int)
df1['is_zero_max_over_limit_count'] = pd.to_numeric(df1['encoded_payment_1'],errors='coerce').astype(int)

In [19]:
df1['is_zero_loans_over_90_days'] = pd.to_numeric(df1['encoded_payment_1'],errors='coerce').astype(int)
df1['is_zero_loans_within_60_to_90_days'] = pd.to_numeric(df1['encoded_payment_23'],errors='coerce').astype(int)
df1['is_zero_loans_within_30_to_60_days'] = pd.to_numeric(df1['encoded_payment_23'],errors='coerce').astype(int)
df1['is_zero_loans_within_5_to_30_days'] = pd.to_numeric(df1['encoded_payment_23'],errors='coerce').astype(int)
df1['is_zero_loans_within_5_days'] = pd.to_numeric(df1['encoded_payment_23'],errors='coerce').astype(int)

In [22]:
y_pred = model.predict(df1)

In [23]:
pred_df = pd.DataFrame(y_pred, columns=['primary_close_flag', 'final_close_flag'])

In [24]:
pred_df.to_csv('prediction.csv', index=False)